# An example file on how to use R within jupyter notebooks
- we first look at importing functions, then how to operate with bigquery

In [ ]:
getwd()
#this will just show you what directory you are in

In [ ]:
# These are the packages you need to install to get pre commits to work... should only need to be done once...
install.packages('docopt')
install.packages('git2r')
install.packages('styler')

In [ ]:
#if you are importing functions into a file outside of fintrans_toolbox you will have to import like this
source('~/fintrans_toolbox/src/example_R_functions.r')


In [ ]:
#example of how to use the function from example_R_functions.R
print_x('boo')

In [ ]:
#example loading in a package
library(data.table) 

In [ ]:
import pandas as pd  # this doesnt work cos its python.

In [ ]:
# hello hello hello hello hello hello more than 88 characters what happpens hellloooooooooooooooooooooooooooooo

In [ ]:
library(bigrquery)
#library(bigQueryR)
library(dplyr)

https://www.infoworld.com/article/3622926/how-to-use-r-with-bigquery.html

This document is the most helpful with how to use R with big query. 

In [ ]:
library(bigrquery)

In [ ]:
con <- dbConnect(
    bigquery(),
    project = 'ons-fintrans-analysis-prod',
    #dataset = 'fintrans_visa'
    )

In [ ]:
dbListTables(con)

In [ ]:
con1 <- dbConnect(
    bigquery(),
    project = 'ons-fintrans-analysis-prod',
    dataset = 'fin_wip_notebook'
    )

In [ ]:
dbListTables(con1)

In [ ]:
sml <- tbl(con1, "sml_cardholders_q")

In [ ]:
str(sml)

In [ ]:
glimpse(sml)

In [ ]:
df <- select(sml, date_time) %>% distinct() %>% collect()

# need to add collect to view things

In [ ]:
df

In [ ]:
select(sml, date_time) %>% distinct() %>% show_query()

In [ ]:
con <- dbConnect(
    bigquery(),
    project = 'ons-fintrans-analysis-prod',
    dataset = 'fintrans_visa'
    )

# Note, we keep the project as 'ons-fintrans-analysis-prod' not 'ons-fintrans-data-prod' even when doing SQLs like the one below
# where we retrieve data from 'ons-fintrans-data-prod'

In [ ]:
con <- dbConnect(
    bigquery(),
    project = 'ons-fintrans-analysis-prod'
    )

# Note, we keep the project as 'ons-fintrans-analysis-prod' not 'ons-fintrans-data-prod' even when doing SQLs like the one below
# where we retrieve data from 'ons-fintrans-data-prod'

In [ ]:
# or running SQL From within R
sql <- "SELECT * FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location LIMIT 10"
library(DBI)
my_results <- dbGetQuery(con, sql)

In [ ]:
my_results

In [ ]:
# or running SQL From within R
sql <- "SELECT * FROM ons-fintrans-analysis-prod.fin_wip_notebook.sml_cardholders_q LIMIT 10"
library(DBI)
my_results <- dbGetQuery(con, sql)
my_results

In [ ]:
#if you are importing functions into a file outside of fintrans_toolbox you will have to import like this
source('~/fintrans_toolbox/src/r_utils.r')

In [ ]:
read_full_bq_table <- function(con, table_id){
    #' Gets data from BigQuery and saves to Pandas DataFrame
    #' 
    #'  Args:
    #'   table_id (str): the table id to determine what table to return
    #'    e.g "ons-fintrans-analysis-prod.fin_wip_notebook.harry_test"
    #' Returns:
    #'    the query results in a Pandas dataframe , or None if error
    

    sql <- paste0("SELECT * FROM ",table_id)
    
    tryCatch(df <- dbGetQuery(con, sql), 
            print("Error getting data"))
    return (df)
}

In [ ]:
table_id ="ons-fintrans-analysis-prod.fin_wip_notebook.sml_cardholders_q"

In [ ]:
paste0("SELECT * FROM ",table_id)

In [ ]:
con <- connect_bigquery()

In [ ]:
connect_bigquery <- function(){
    con <- dbConnect(
    bigquery(),
    project = 'ons-fintrans-analysis-prod',
    dataset = 'fintrans_visa'
    )
    
    return (con)
}

In [ ]:

read_full_bq_table <- function(con, table_id){
    #' Gets data from BigQuery and saves to Pandas DataFrame
    #' 
    #'  Args:
    #'   table_id (str): the table id to determine what table to return
    #'    e.g "ons-fintrans-analysis-prod.fin_wip_notebook.harry_test"
    #' Returns:
    #'    the query results in a Pandas dataframe , or None if error
    

    sql <- paste0("SELECT * FROM ",table_id)
    print(sql)
    try(df <- dbGetQuery(con, sql))
    return (df)
}

In [ ]:
df <- read_full_bq_table(con, "ons-fintrans-analysis-prod.fin_wip_notebook.sml_cardholders_q")

In [ ]:
df

In [ ]:
read_bq_table_sql <- function(con, sql){
    #' Gets data from BigQuery and saves to Pandas DataFrame
    #' 
    #'  Args:
    #'   table_id (str): the table id to determine what table to return
    #'    e.g "ons-fintrans-analysis-prod.fin_wip_notebook.harry_test"
    #' Returns:
    #'    the query results in a Pandas dataframe , or None if error

    print(sql)
    try(df <- dbGetQuery(con, sql))
    return (df)
}

In [ ]:
read_spend_merchant_location <- function(
    con,
    time_period="Quarter",
    merchant_location_level="All",
    cardholder_issuing_level="All",
    mcg="All",
    mcc="All",
    merchant_location="",
    cardholder_issuing_country=""
){
    
    if (merchant_location != ""){
        SQL2 = paste0(" AND merchant_location IN (",merchant_location, ")")
        merchant_location_level = ""
    }
    else {
        SQL2 = ""
    }

    if (cardholder_issuing_country != ""){
        SQL3 = paste0(" AND cardholder_issuing_country IN (",cardholder_issuing_country,")")
        cardholder_issuing_level = ""
    }
    else {
        SQL3 = ""
    }
    if (time_period == "Quarter"){
        SQL_date = "DATETIME( CAST(SUBSTRING(time_period_value, 1,4) AS int),
        CAST(SUBSTRING(time_period_value,6,6)AS int)*3, 01,00,00,00) AS
        date_time, "
    }
    else if (time_period == "Month"){
        SQL_date = "DATETIME( CAST(SUBSTRING(time_period_value, 1,4) AS int),
        CAST(SUBSTRING(time_period_value,5,6)AS int), 01,00,00,00) AS
        date_time, "
    }
    else {
        SQL_date = " "
    }

    SQL1 = paste0("
        SELECT ",SQL_date," * , FROM
        ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
        WHERE
        time_period LIKE '",time_period,"%' AND
        cardholder_issuing_level LIKE '",cardholder_issuing_level,"%' AND
        merchant_location_level LIKE '",merchant_location_level,"%' AND
        mcg LIKE '",mcg,"%' AND
        mcc LIKE '",mcc,"%'")

    SQL4 = " ORDER BY time_period, time_period_value"

    SQL = paste0(SQL1, SQL2,SQL3,SQL4)

    print(paste0("The SQL statement you have just selected is: ",SQL))

    df <- read_bq_table_sql(con, SQL)

    return (df)
}

In [ ]:
df <- read_spend_merchant_location(con)

In [ ]:
df

In [ ]:
#if you are importing functions into a file outside of fintrans_toolbox you will have to import like this
source('~/fintrans_toolbox/src/r_utils.r')

In [ ]:
df <- read_spend_merchant_location(con, merchant_location_level ="POSTAL_SECTOR")
df